Normalize the data

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler


def load_and_normalize_data(file_path):
    data = pd.read_csv(file_path, header=None)

    # Normalize the data using z-score
    scaler = StandardScaler()
    normalized_data = scaler.fit_transform(data)

    return normalized_data

#give the required file name to be normalized
file_names = ['1.txt']


common_data = pd.DataFrame()

for file_name in file_names:
    file_path = f'/content/drive/MyDrive/S-1/Test/{file_name}'  # Replace with the actual path
    normalized_data = load_and_normalize_data(file_path)
    common_data = common_data.append(pd.DataFrame(normalized_data), ignore_index=True)

# change the name to required file name after normalization
common_data.to_csv('/content/drive/MyDrive/S-1/Normal data/S1_2_test_normal.csv', index=False)

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Threshold Calculation

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, RepeatVector, TimeDistributed, Dense


def load_and_preprocess_data(file_path, time_steps=10):
    data = pd.read_csv(file_path, header=None)
    return np.array(data)


def calculate_threshold(X_train, model, percentile=95):

    reconstructions = model.predict(X_train)
    mse = np.mean(np.square(X_train - reconstructions), axis=(1, 2))


    threshold = np.percentile(mse, percentile)
    return threshold


def train_lstm_autoencoder(X_train, epochs=10, batch_size=32):
    model = Sequential()
    model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(RepeatVector(X_train.shape[1]))
    model.add(LSTM(units=64, return_sequences=True))
    model.add(TimeDistributed(Dense(X_train.shape[2])))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, X_train, epochs=epochs, batch_size=batch_size)
    return model




# give the correct normalized train file path
train_file_path = r'/content/drive/MyDrive/S-1/Normal data/S1_3_train_normal.csv'
X_train = load_and_preprocess_data(train_file_path)


X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))


lstm_autoencoder_model = train_lstm_autoencoder(X_train)


threshold = calculate_threshold(X_train, lstm_autoencoder_model)

print(f"Threshold: {threshold}")

Epoch 1/10
2372/2372 [==============================] - 136s 54ms/step - loss: 0.5599
Epoch 2/10
2372/2372 [==============================] - 127s 54ms/step - loss: 0.5080
Epoch 3/10
2372/2372 [==============================] - 126s 53ms/step - loss: 0.4742
Epoch 4/10
2372/2372 [==============================] - 125s 53ms/step - loss: 0.4320
Epoch 5/10
2372/2372 [==============================] - 125s 52ms/step - loss: 0.3955
Epoch 6/10
2372/2372 [==============================] - 126s 53ms/step - loss: 0.3968
Epoch 7/10
2372/2372 [==============================] - 132s 56ms/step - loss: 0.3269
Epoch 8/10
2372/2372 [==============================] - 133s 56ms/step - loss: 0.2939
Epoch 9/10
2372/2372 [==============================] - 131s 55ms/step - loss: 0.2743
Epoch 10/10
2372/2372 [==============================] - 45s 19ms/step
Threshold: 0.44315840680685453


In [ ]:
from tensorflow.keras.layers import LSTM
import pandas as pd

def save_predicted_labels(labels, output_path):
    df = pd.DataFrame({'Anomaly': labels})
    df.to_csv(output_path, index=False)



def load_and_preprocess_data(file_path, time_steps=10):
    data = pd.read_csv(file_path, header=None)


    X, y = [], []
    for i in range(len(data) - time_steps + 1):
        X.append(data.values[i:i + time_steps, :])
        y.append(data.values[i + time_steps - 1, :])

    return np.array(X), np.array(y)


def detect_anomalies(model, X_test, threshold=0.5):

    X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])

    reconstructions = model.predict(X_test_reshaped)


    mse = np.mean(np.square(X_test_reshaped - reconstructions), axis=(1, 2))


    anomalies = (mse > threshold).astype(int)

    return anomalies


def train_lstm_autoencoder(X_train, epochs=10, batch_size=32):
    model = Sequential()
    model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(RepeatVector(X_train.shape[1]))
    model.add(LSTM(units=64, return_sequences=True))
    model.add(TimeDistributed(Dense(X_train.shape[2])))

    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, X_train, epochs=epochs, batch_size=batch_size)

    return model

# give the correct normalized train file path
train_file_path = r'/content/drive/MyDrive/S-1/Normal data/S1_3_train_normal.csv'
X_train, y_train = load_and_preprocess_data(train_file_path)

# give the correct normalized test file path
test_file_path = r'/content/drive/MyDrive/S-1/Normal data/S1_3_test_normal.csv'
X_test, y_test = load_and_preprocess_data(test_file_path)


lstm_autoencoder_model = train_lstm_autoencoder(X_train)


anomalies = detect_anomalies(lstm_autoencoder_model, X_test, threshold=0.443)  # Adjust threshold

# Save predicted labels to a CSV file
output_csv_path = r'/content/drive/MyDrive/S-1/Pre_lab/3_predicted_labels_lstm.csv'
save_predicted_labels(anomalies, output_csv_path)




Epoch 1/5
237/237 [==============================] - 18s 58ms/step - loss: 0.3399
Epoch 2/5
237/237 [==============================] - 10s 41ms/step - loss: 0.2436
Epoch 3/5
237/237 [==============================] - 13s 53ms/step - loss: 0.2190
Epoch 4/5
237/237 [==============================] - 13s 57ms/step - loss: 0.2064
Epoch 5/5
237/237 [==============================] - 10s 41ms/step - loss: 0.1995


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


741/741 [==============================] - 8s 9ms/step


Performance Measures

In [ ]:
import csv

def count_ones_and_matches(file1_path, file2_path):
    ones_count_file1 = 0
    zero_count_file1 = 0
    zero_count_file2 = 0
    ones_count_file2 = 0
    correct_matches_count_1 = 0
    correct_matches_count_0 = 0

    with open(file1_path, 'r') as file1, open(file2_path, 'r') as file2:
        csv_reader1 = csv.reader(file1)
        csv_reader2 = csv.reader(file2)


        csv_data1 = [row for row in csv_reader1]
        csv_data2 = [row for row in csv_reader2]


    for row in csv_data1:
        ones_count_file1 += row.count('1')
        zero_count_file1 += row.count('0')

    for row in csv_data2:
        ones_count_file2 += row.count('1')
        zero_count_file2 += row.count('0')


    for row1, row2 in zip(csv_data1, csv_data2):
        correct_matches_count_1 += sum(1 for a, b in zip(row1, row2) if a == b == '1')
        correct_matches_count_0 += sum(1 for a, b in zip(row1, row2) if a == b == '0')

    precision = correct_matches_count_0 / zero_count_file1
    recall = correct_matches_count_0 / (correct_matches_count_0 + (ones_count_file1 - correct_matches_count_1))
    f1_score = (2 * precision * recall) / (precision + recall)


    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 score: {f1_score}")

    return precision, recall, f1_score


average_precision = 0
average_recall = 0
average_f1_score = 0

for i in range(1, 8):
    # give the predicted labels file path
    csv_file1_path = f'/content/drive/MyDrive/S-1/Pre_lab/{i}_predicted_labels_lstm.csv'
    # give the actual labels file path
    csv_file2_path = f'/content/drive/MyDrive/S-1/Test_Label/{i}.csv'

    print(f"\nEvaluation for iteration {i}:")
    precision, recall, f1_score = count_ones_and_matches(csv_file1_path, csv_file2_path)

    average_precision += precision
    average_recall += recall
    average_f1_score += f1_score

#change the denominator to number of files in that particular set
average_precision /= 7
average_recall /= 7
average_f1_score /= 7

print("\nAverage Precision:", average_precision)
print("Average Recall:", average_recall)
print("Average F1 score:", average_f1_score)



Evaluation for iteration 1:
Precision: 0.9229162916291629
Recall: 0.9944524188229817
F1 score: 0.9573498655512399

Evaluation for iteration 2:
Precision: 0.9869449378330373
Recall: 0.9603352920843415
F1 score: 0.9734583041345479

Evaluation for iteration 3:
Precision: 0.9660979877515311
Recall: 0.9653800760589238
F1 score: 0.9657388984848154

Evaluation for iteration 4:
Precision: 0.9697442566103164
Recall: 0.9736269475150143
F1 score: 0.9716817234190409

Evaluation for iteration 5:
Precision: 0.996728992618132
Recall: 0.9555875746916981
F1 score: 0.9757247944612722

Evaluation for iteration 6:
Precision: 0.8466141593699125
Recall: 0.9633987582615662
F1 score: 0.9012389048923861

Evaluation for iteration 7:
Precision: 0.9556053401609363
Recall: 0.9816824010145132
F1 score: 0.9684683641081483

Average Precision: 0.949235995139004
Average Recall: 0.9706376383498627
Average F1 score: 0.959094407864493
